In [40]:
# =============================================================================
# SETUP: Initialize the OpenAI Client
# =============================================================================
# We import the OpenAI library and create a client instance.
# The client automatically uses the OPENAI_API_KEY environment variable.
# This single client will be used for all API calls in this notebook.

import os
from openai import OpenAI
import json  # Used to parse JSON responses from the model

# Initialize the client - this connects to the OpenAI API
# Make sure your API key is set: export OPENAI_API_KEY="your-key-here"
client = OpenAI()

print("Client initialized successfully.")

Client initialized successfully.


---
## Pattern 1: Fast Path, Then Escalate

**Strategy:** Start with low reasoning effort for quick/cheap responses, then escalate to high effort when the task requires deep thinking.

**When it's good:**
- Mixed difficulty workloads (some easy, some complex)
- User-facing apps where speed matters for simple requests
- Cost optimization: save money on easy tasks, invest in hard ones

**Key benefit:** Pay for deep thinking only when the task truly needs it.

### Example: Simple vs Complex Tasks

Some tasks are simple (classification, lookups) while others require multi-step reasoning (math problems, logic puzzles, analysis). We'll demonstrate the difference by comparing low vs high effort on a problem that truly benefits from reasoning.

In [41]:
# =============================================================================
# PATTERN 1: Fast Path, Then Escalate
# =============================================================================
# This pattern saves money by using low effort for simple tasks, and only
# "escalating" to high effort when the task requires deep reasoning.
#
# KEY INSIGHT: Not all tasks trigger reasoning tokens!
# - Classification, simple lookups → No reasoning needed (even at "high" effort)
# - Math problems, logic puzzles, analysis → These benefit from reasoning
#
# The model itself decides when to "think deeply" based on task complexity.
# We'll demonstrate with a math problem that clearly shows the difference.

print("Pattern 1 setup complete.")
print("\nKey concept: Escalate from low→high effort only for tasks that")
print("require multi-step reasoning (math, logic, analysis).")

Pattern 1 setup complete.

Key concept: Escalate from low→high effort only for tasks that
require multi-step reasoning (math, logic, analysis).


In [52]:
# =============================================================================
# PATTERN 1: Escalation Function with Routing Logic
# =============================================================================
# This function implements the ACTUAL escalation pattern:
#   1. Try LOW effort first (fast/cheap)
#   2. Check if the response indicates uncertainty
#   3. If uncertain → ESCALATE to HIGH effort
#   4. If confident → Return the low-effort response (saved money!)
#
# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ THIS IS THE KEY CONCEPT: ROUTING + ESCALATION                               │
# │                                                                             │
# │ ROUTING = Deciding which path to take based on criteria                     │
# │ ESCALATION = Moving from cheap→expensive only when needed                   │
# │                                                                             │
# │ The magic is in the "should we escalate?" decision logic.                   │
# └─────────────────────────────────────────────────────────────────────────────┘

def solve_with_escalation(problem):
    """
    Solve a problem using the escalation pattern.
    
    Step 1: Try LOW effort first (fast and cheap)
    Step 2: Check if the model expressed uncertainty
    Step 3: If uncertain, ESCALATE to HIGH effort
    Step 4: Return the best response
    """
    
    print("┌" + "─"*58 + "┐")
    print("│ STEP 1: Try LOW effort first (fast/cheap)                │")
    print("└" + "─"*58 + "┘")
    
    # -------------------------------------------------------------------------
    # FIRST ATTEMPT: Low effort (the "fast path")
    # -------------------------------------------------------------------------
    low_response = client.responses.create(
        model="gpt-5.2",
        input=[{"role": "user", "content": f"""{problem}

At the end of your response, rate your confidence: [HIGH], [MEDIUM], or [LOW].
If you're uncertain about any step, say [LOW]."""}],
        reasoning={"effort": "low"}  # 👈 Start cheap
    )
    
    low_answer = low_response.output_text
    low_tokens = low_response.usage.output_tokens_details.reasoning_tokens
    
    print(f"   Reasoning tokens used: {low_tokens}")
    print(f"   Response preview: {low_answer[:200]}...")
    
    # -------------------------------------------------------------------------
    # ROUTING DECISION: Should we escalate?
    # -------------------------------------------------------------------------
    # Look for confidence indicators in the response
    # This is a simple heuristic - in production you might use more sophisticated checks
    
    print("\n┌" + "─"*58 + "┐")
    print("│ STEP 2: ROUTING DECISION - Should we escalate?           │")
    print("└" + "─"*58 + "┘")
    
    needs_escalation = (
        "[LOW]" in low_answer or 
        "[MEDIUM]" in low_answer or
        "uncertain" in low_answer.lower() or
        "not sure" in low_answer.lower()
    )
    
    if needs_escalation:
        print("   🔍 Detected uncertainty in response!")
        print("   📈 DECISION: ESCALATE to high effort")
        
        # ---------------------------------------------------------------------
        # ESCALATION: Use HIGH effort for a more thorough answer
        # ---------------------------------------------------------------------
        print("\n┌" + "─"*58 + "┐")
        print("│ STEP 3: ESCALATING to HIGH effort (thorough/expensive)  │")
        print("└" + "─"*58 + "┘")
        
        high_response = client.responses.create(
            model="gpt-5.2",
            input=[{"role": "user", "content": problem}],
            reasoning={"effort": "high"}  # 👈 Escalate to thorough
        )
        
        high_tokens = high_response.usage.output_tokens_details.reasoning_tokens
        print(f"   Reasoning tokens used: {high_tokens}")
        print(f"   Additional cost: {high_tokens - low_tokens} more reasoning tokens")
        
        return {
            "escalated": True,
            "low_response": low_response,
            "high_response": high_response,
            "final_answer": high_response.output_text
        }
    else:
        print("   ✅ Response appears confident!")
        print("   💰 DECISION: KEEP low-effort response (saved money!)")
        
        return {
            "escalated": False,
            "low_response": low_response,
            "high_response": None,
            "final_answer": low_answer
        }

print("Escalation function defined!")
print("\n💡 This function implements ROUTING + ESCALATION:")
print("   1. Always tries LOW effort first")
print("   2. Checks for uncertainty signals")
print("   3. Only escalates to HIGH if needed")

Escalation function defined!

💡 This function implements ROUTING + ESCALATION:
   1. Always tries LOW effort first
   2. Checks for uncertainty signals
   3. Only escalates to HIGH if needed


In [58]:
# =============================================================================
# DEMO 1: A Problem That WILL Trigger Escalation
# =============================================================================
# This problem has MISSING INFORMATION - the model cannot be fully confident.
# It MUST express uncertainty, which triggers our escalation logic.

complex_problem = """
A company's server crashed. The logs show:
- 10:00 AM: Memory usage spiked to 95%
- 10:02 AM: Database connection pool exhausted  
- 10:03 AM: Server became unresponsive

What was the root cause of the crash? 
Was it a memory leak, a database issue, or something else?

Note: You don't have access to the code or additional logs.
"""

print("="*60)
print("TEST 1: Ambiguous Problem (Missing Information)")
print("="*60)
print(f"Problem: {complex_problem[:100]}...")

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CALL THE ESCALATION FUNCTION                                                │
# │                                                                             │
# │ This problem has INCOMPLETE DATA - the model should express uncertainty.    │
# │ Our routing logic will detect this and ESCALATE to high effort.             │
# └─────────────────────────────────────────────────────────────────────────────┘
result_complex = solve_with_escalation(complex_problem)

print("\n" + "="*60)
print("RESULT:")
print("="*60)
print(f"Was escalation needed? {'YES ⬆️' if result_complex['escalated'] else 'NO 💰'}")
print(f"\nFinal Answer:\n{result_complex['final_answer']}")

TEST 1: Ambiguous Problem (Missing Information)
Problem: 
A company's server crashed. The logs show:
- 10:00 AM: Memory usage spiked to 95%
- 10:02 AM: Datab...
┌──────────────────────────────────────────────────────────┐
│ STEP 1: Try LOW effort first (fast/cheap)                │
└──────────────────────────────────────────────────────────┘
   Reasoning tokens used: 67
   Response preview: From the timeline alone, the most defensible conclusion is:

**The immediate/root technical trigger was extreme memory pressure (95% at 10:00), which likely caused system-wide degradation and then cas...

┌──────────────────────────────────────────────────────────┐
│ STEP 2: ROUTING DECISION - Should we escalate?           │
└──────────────────────────────────────────────────────────┘
   🔍 Detected uncertainty in response!
   📈 DECISION: ESCALATE to high effort

┌──────────────────────────────────────────────────────────┐
│ STEP 3: ESCALATING to HIGH effort (thorough/expensive)  │
└─────────────────

In [54]:
# =============================================================================
# DEMO 2: A Simple Problem That Should NOT Escalate
# =============================================================================
# This is a simple factual question. The model should be confident
# and we should save money by NOT escalating.

simple_problem = """
What is the capital of France? Just give me the answer.
"""

print("="*60)
print("TEST 2: Simple Factual Question")
print("="*60)
print(f"Problem: {simple_problem.strip()}")

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CALL THE SAME ESCALATION FUNCTION                                           │
# │                                                                             │
# │ For this simple question, the routing logic should decide:                  │
# │   "Response is confident → NO escalation needed → Save money!"              │
# └─────────────────────────────────────────────────────────────────────────────┘
result_simple = solve_with_escalation(simple_problem)

print("\n" + "="*60)
print("RESULT:")
print("="*60)
print(f"Was escalation needed? {'YES ⬆️' if result_simple['escalated'] else 'NO 💰 (saved money!)'}")
print(f"\nFinal Answer:\n{result_simple['final_answer']}")

TEST 2: Simple Factual Question
Problem: What is the capital of France? Just give me the answer.
┌──────────────────────────────────────────────────────────┐
│ STEP 1: Try LOW effort first (fast/cheap)                │
└──────────────────────────────────────────────────────────┘
   Reasoning tokens used: 0
   Response preview: Paris. [HIGH]...

┌──────────────────────────────────────────────────────────┐
│ STEP 2: ROUTING DECISION - Should we escalate?           │
└──────────────────────────────────────────────────────────┘
   ✅ Response appears confident!
   💰 DECISION: KEEP low-effort response (saved money!)

RESULT:
Was escalation needed? NO 💰 (saved money!)

Final Answer:
Paris. [HIGH]


In [59]:
# =============================================================================
# PATTERN 1 SUMMARY: Escalation & Routing Results
# =============================================================================
# Let's compare what happened with each problem.

print("="*60)
print("💰 ESCALATION & ROUTING SUMMARY")
print("="*60)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ THE KEY INSIGHT: ROUTING SAVES MONEY                                        │
# │                                                                             │
# │ - Simple questions → Low effort is enough → No escalation → Cheap!          │
# │ - Complex problems → Uncertainty detected → Escalate → More accurate        │
# │                                                                             │
# │ Without routing, you'd use HIGH effort on everything (expensive!)           │
# │ With routing, you only pay for HIGH effort when it's truly needed.          │
# └─────────────────────────────────────────────────────────────────────────────┘

print("\n📊 COMPARISON:")
print("-"*60)

# Complex problem stats
complex_low_tokens = result_complex['low_response'].usage.output_tokens_details.reasoning_tokens
complex_total = result_complex['low_response'].usage.output_tokens
if result_complex['escalated'] and result_complex['high_response']:
    complex_high_tokens = result_complex['high_response'].usage.output_tokens_details.reasoning_tokens
    complex_total += result_complex['high_response'].usage.output_tokens

print(f"\n🧮 Complex Math Problem:")
print(f"   Escalated: {'YES' if result_complex['escalated'] else 'NO'}")
print(f"   Low effort reasoning tokens: {complex_low_tokens}")
if result_complex['escalated'] and result_complex['high_response']:
    print(f"   High effort reasoning tokens: {complex_high_tokens}")
    print(f"   Total tokens used: {complex_total}")

# Simple problem stats  
simple_low_tokens = result_simple['low_response'].usage.output_tokens_details.reasoning_tokens
simple_total = result_simple['low_response'].usage.output_tokens

print(f"\n❓ Simple Factual Question:")
print(f"   Escalated: {'YES' if result_simple['escalated'] else 'NO'}")
print(f"   Low effort reasoning tokens: {simple_low_tokens}")
print(f"   Total tokens used: {simple_total}")
if not result_simple['escalated']:
    print(f"   💰 SAVED by not escalating!")

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ KEY TAKEAWAYS                                                               │
# └─────────────────────────────────────────────────────────────────────────────┘
print("\n" + "="*60)
print("🎯 KEY TAKEAWAYS")
print("="*60)
print("""
1. ROUTING = Deciding which path based on response analysis
   We checked for [LOW], [MEDIUM], "uncertain" in the response

2. ESCALATION = Moving from cheap → expensive when needed
   Only pay for high effort when the task truly requires it

3. COST SAVINGS = Simple questions never escalate
   You save money on every simple question that stays on the fast path

4. REAL-WORLD APPLICATION:
   • Customer support: Simple FAQs stay low, complex issues escalate
   • Code review: Syntax questions stay low, logic bugs escalate
   • Analysis: Lookups stay low, multi-factor analysis escalates
""")

💰 ESCALATION & ROUTING SUMMARY

📊 COMPARISON:
------------------------------------------------------------

🧮 Complex Math Problem:
   Escalated: YES
   Low effort reasoning tokens: 67
   High effort reasoning tokens: 110
   Total tokens used: 1291

❓ Simple Factual Question:
   Escalated: NO
   Low effort reasoning tokens: 0
   Total tokens used: 9
   💰 SAVED by not escalating!

🎯 KEY TAKEAWAYS

1. ROUTING = Deciding which path based on response analysis
   We checked for [LOW], [MEDIUM], "uncertain" in the response

2. ESCALATION = Moving from cheap → expensive when needed
   Only pay for high effort when the task truly requires it

3. COST SAVINGS = Simple questions never escalate
   You save money on every simple question that stays on the fast path

4. REAL-WORLD APPLICATION:
   • Customer support: Simple FAQs stay low, complex issues escalate
   • Code review: Syntax questions stay low, logic bugs escalate
   • Analysis: Lookups stay low, multi-factor analysis escalates



---
## Pattern 2: Always Reason for High-Risk Workflows

**Strategy:** Route specific critical endpoints directly to reasoning models. No escalation needed—always use deep reasoning.

**When it's good:**
- Compliance and legal review
- Financial decisions
- Planning and scheduling
- Multi-step code changes

**Key benefit:** Predictable cost. Predictable latency.

### Example: Financial Decision Analysis

For financial advice, we always want thorough reasoning. The stakes are too high for quick answers.

In [46]:
# =============================================================================
# PATTERN 2: Always Reason for High-Risk Workflows
# =============================================================================
# For critical decisions (financial, legal, compliance), we ALWAYS use high
# reasoning effort. The cost of a mistake outweighs the cost of extra tokens.
#
# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ WHY "ALWAYS HIGH" FOR FINANCIAL DECISIONS?                                  │
# │                                                                             │
# │ Imagine you're a bank deciding whether to approve a loan.                   │
# │ A wrong decision could cost thousands of dollars.                           │
# │ The extra $0.01 in reasoning tokens is worth it for accuracy!               │
# │                                                                             │
# │ Key insight: "Predictable cost. Predictable quality."                       │
# │ - Every request uses high effort (no surprises)                             │
# │ - Easier to budget for                                                      │
# │ - Consistent quality on every response                                      │
# └─────────────────────────────────────────────────────────────────────────────┘

def analyze_financial_decision(scenario):
    """
    Analyze a financial scenario using ALWAYS HIGH reasoning effort.
    
    This function wraps the API call so we don't accidentally use low effort
    for important decisions. The high effort is "baked in" to the function.
    """
    
    # ┌─────────────────────────────────────────────────────────────────────────┐
    # │ THE API CALL - Notice effort is HARDCODED to "high"                     │
    # │                                                                         │
    # │ We don't give callers the option to use low effort here.                │
    # │ This is intentional - financial decisions should ALWAYS be thorough.    │
    # └─────────────────────────────────────────────────────────────────────────┘
    response = client.responses.create(
        model="gpt-5.2",
        
        # The prompt asks for structured JSON output
        # This makes it easier to parse and use in our application
        input=[{"role": "user", "content": f"""You are a financial analyst. Analyze this scenario thoroughly.

Scenario: {scenario}

Provide your analysis in JSON format:
{{
    "recommendation": "approve|deny|review",
    "risk_level": "low|medium|high",
    "key_factors": ["factor1", "factor2", ...],
    "concerns": ["concern1", "concern2", ...],
    "suggested_conditions": ["condition1", ...]
}}

Just the JSON, no markdown."""}],
        
        # ┌─────────────────────────────────────────────────────────────────────┐
        # │ 👇 THIS IS THE KEY LINE - Always "high" for financial decisions     │
        # └─────────────────────────────────────────────────────────────────────┘
        reasoning={"effort": "high"}
    )
    
    return response

print("Financial analysis function defined (always uses high reasoning).")
print("\n💡 Notice: The function ALWAYS uses effort='high' - no option for 'low'")
print("   This prevents accidental use of fast/cheap mode for important decisions.")

Financial analysis function defined (always uses high reasoning).


In [ ]:
# =============================================================================
# EXECUTE: Financial Analysis with High Reasoning
# =============================================================================
# Let's run the financial analysis function on a realistic loan scenario.
# This scenario has MIXED signals - some good, some concerning.
# A thorough analysis should catch the nuances.

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ THE TEST SCENARIO - Notice the mixed signals:                               │
# │                                                                             │
# │ GOOD SIGNS:                                                                 │
# │   ✓ Revenue growing 10% month-over-month                                    │
# │   ✓ Owner has decent credit (720)                                           │
# │   ✓ Has collateral ($30k equipment)                                         │
# │                                                                             │
# │ CONCERNING SIGNS:                                                           │
# │   ✗ Business is only 18 months old                                          │
# │   ✗ Already has $15k debt                                                   │
# │   ✗ Restaurant industry is risky                                            │
# │                                                                             │
# │ A good analysis should weigh ALL of these factors.                          │
# └─────────────────────────────────────────────────────────────────────────────┘
financial_scenario = """
A small business is requesting a $50,000 line of credit.
- Business age: 18 months
- Monthly revenue: $25,000 (growing 10% month-over-month)
- Current debt: $15,000 equipment loan
- Owner credit score: 720
- Industry: Restaurant (post-pandemic recovery)
- Collateral: Restaurant equipment valued at $30,000
"""

print("="*60)
print("HIGH-RISK WORKFLOW: Financial Decision (Always High Effort)")
print("="*60)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CALL THE FUNCTION - Remember, it always uses effort="high" internally       │
# └─────────────────────────────────────────────────────────────────────────────┘
response_financial = analyze_financial_decision(financial_scenario)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CHECK THE TOKEN USAGE                                                       │
# │                                                                             │
# │ Because we used effort="high", we expect to see a significant number of     │
# │ reasoning tokens. This is the model "thinking through" the decision.        │
# └─────────────────────────────────────────────────────────────────────────────┘
print(f"\nReasoning tokens used: {response_financial.usage.output_tokens_details.reasoning_tokens}")
print(f"Total output tokens: {response_financial.usage.output_tokens}")

# The analysis - should include multiple factors, concerns, and conditions
print(f"\nAnalysis:\n{response_financial.output_text}")

HIGH-RISK WORKFLOW: Financial Decision (Always High Effort)

Reasoning tokens used: 111
Total output tokens: 620

Analysis:
{
  "recommendation": "review",
  "risk_level": "medium",
  "key_factors": [
    "Strong owner credit score (720) supports willingness/ability to repay",
    "Monthly revenue of $25,000 with ~10% month-over-month growth indicates improving demand and traction",
    "Business has operated 18 months, showing some operating history beyond initial launch",
    "Existing term debt is moderate ($15,000 equipment loan) and likely supported by existing assets",
    "Collateral available (restaurant equipment valued at ~$30,000) can partially secure exposure"
  ],
  "concerns": [
    "Restaurant industry carries elevated volatility (seasonality, thin margins, labor/food cost inflation) despite post-pandemic recovery",
    "Limited operating history (18 months) reduces confidence in performance through multiple cycles",
    "Collateral coverage is partial: $30,000 equipment

---
## Pattern 3: Draft, Then Verify

**Strategy:** Use a standard model (or low reasoning) to generate a draft quickly, then use a reasoning model to verify and fix it.

**When it's good:**
- Content generation
- Code review
- Summaries and explanations

**Key benefit:** Reasoning as a safety net.

### Example: Code Generation with Verification

Generate code quickly with low effort, then verify it with high effort reasoning.

In [ ]:
# =============================================================================
# PATTERN 3: Draft, Then Verify
# =============================================================================
# This is a two-step pattern that combines the best of both worlds:
#
# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ THE TWO-STEP APPROACH:                                                      │
# │                                                                             │
# │ STEP 1: DRAFT (Low Effort)                                                  │
# │   → Generate content quickly                                                │
# │   → Fast and cheap                                                          │
# │   → Might have bugs or miss edge cases                                      │
# │                                                                             │
# │ STEP 2: VERIFY (High Effort)                                                │
# │   → Review the draft carefully                                              │
# │   → Catch bugs and edge cases                                               │
# │   → Slower and more expensive, but thorough                                 │
# │                                                                             │
# │ WHY THIS WORKS:                                                             │
# │   Creating content from scratch is "creative" - low effort is fine          │
# │   Reviewing for bugs is "analytical" - high effort catches more             │
# │   Total cost is less than doing everything at high effort                   │
# └─────────────────────────────────────────────────────────────────────────────┘

# =============================================================================
# FUNCTION 1: Generate a Draft (Low Effort)
# =============================================================================
def generate_code_draft(task):
    """
    Generate code quickly using LOW reasoning effort.
    Speed matters more than perfection here - we'll review it next.
    """
    
    response = client.responses.create(
        model="gpt-5.2",
        
        input=[{"role": "user", "content": f"""Write Python code for this task:

{task}

Provide only the code, no explanations."""}],
        
        # ┌─────────────────────────────────────────────────────────────────────┐
        # │ 👇 LOW EFFORT for drafts - optimize for speed                       │
        # │    We're not worried about perfection here.                         │
        # │    The verification step will catch any issues.                     │
        # └─────────────────────────────────────────────────────────────────────┘
        reasoning={"effort": "low"}
    )
    
    return response


# =============================================================================
# FUNCTION 2: Verify and Fix the Draft (High Effort)
# =============================================================================
def verify_and_fix_code(code, original_task):
    """
    Verify code using HIGH reasoning effort.
    This is where we catch bugs, edge cases, and suggest improvements.
    """
    
    response = client.responses.create(
        model="gpt-5.2",
        
        input=[{"role": "user", "content": f"""Review this code carefully for bugs, edge cases, and improvements.

Original task: {original_task}

Code to review:
```python
{code}
```

Respond in JSON format:
{{
    "has_issues": true/false,
    "issues_found": ["issue1", "issue2", ...],
    "fixed_code": "corrected code here",
    "improvements_made": ["improvement1", ...]
}}

Just the JSON, no markdown formatting."""}],
        
        # ┌─────────────────────────────────────────────────────────────────────┐
        # │ 👇 HIGH EFFORT for verification - thoroughness matters here         │
        # │    This is where we want the model to think carefully.              │
        # │    Catching a bug now is cheaper than finding it in production!     │
        # └─────────────────────────────────────────────────────────────────────┘
        reasoning={"effort": "high"}
    )
    
    return response

print("Draft and verify functions defined.")
print("\n💡 Notice the pattern:")
print("   • generate_code_draft() uses effort='low'  (fast draft)")
print("   • verify_and_fix_code() uses effort='high' (thorough review)")

Draft and verify functions defined.


In [ ]:
# =============================================================================
# STEP 1: Generate the Draft (Low Effort)
# =============================================================================
# We'll ask for code that has a potential edge case: division.
# What happens if divisor is 0? What if the list is empty?
# A quick low-effort draft might miss these edge cases. Let's see!

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ THE TASK - Notice the hidden edge cases:                                    │
# │                                                                             │
# │ "Divide each number by the divisor"                                         │
# │                                                                             │
# │ Edge cases that might be missed:                                            │
# │   1. What if divisor is 0? (Division by zero error)                         │
# │   2. What if the list is empty?                                             │
# │   3. What if the list contains non-numbers?                                 │
# │                                                                             │
# │ A quick draft might just do the division without checking these!            │
# └─────────────────────────────────────────────────────────────────────────────┘
coding_task = """
Write a function called 'divide_list' that takes a list of numbers and a divisor.
Return a new list with each number divided by the divisor.
"""

print("="*60)
print("STEP 1: Generate Draft (Low Effort)")
print("="*60)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CALL THE DRAFT FUNCTION - Uses effort="low" internally                      │
# └─────────────────────────────────────────────────────────────────────────────┘
draft_response = generate_code_draft(coding_task)

# Save the code for the verification step
draft_code = draft_response.output_text

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CHECK REASONING TOKENS - Should be LOW or ZERO                              │
# │                                                                             │
# │ With effort="low", the model doesn't think too hard.                        │
# │ It generates code quickly but might miss edge cases.                        │
# └─────────────────────────────────────────────────────────────────────────────┘
print(f"Reasoning tokens: {draft_response.usage.output_tokens_details.reasoning_tokens}")
print(f"\n💡 Low/zero reasoning tokens = fast and cheap, but might have bugs")

# Show the draft code - does it handle division by zero?
print(f"\nDraft Code:\n{draft_code}")

STEP 1: Generate Draft (Low Effort)
Reasoning tokens: 0

Draft Code:
```python
def divide_list(numbers, divisor):
    return [num / divisor for num in numbers]
```


In [ ]:
# =============================================================================
# STEP 2: Verify and Fix (High Effort)
# =============================================================================
# Now we pass the draft to our HIGH effort verifier.
# This is where we catch the bugs that the quick draft might have missed.

print("\n" + "="*60)
print("STEP 2: Verify and Fix (High Effort)")
print("="*60)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CALL THE VERIFY FUNCTION - Uses effort="high" internally                    │
# │                                                                             │
# │ We pass in:                                                                 │
# │   1. draft_code - the code from Step 1                                      │
# │   2. coding_task - the original task (for context)                          │
# │                                                                             │
# │ The model will carefully review the code and look for:                      │
# │   - Bugs (like missing error handling)                                      │
# │   - Edge cases (division by zero, empty lists)                              │
# │   - Potential improvements                                                  │
# └─────────────────────────────────────────────────────────────────────────────┘
verify_response = verify_and_fix_code(draft_code, coding_task)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CHECK REASONING TOKENS - Should be MUCH HIGHER                              │
# │                                                                             │
# │ With effort="high", the model thinks carefully about edge cases.            │
# │ This costs more but catches bugs that would be expensive in production.     │
# └─────────────────────────────────────────────────────────────────────────────┘
print(f"Reasoning tokens: {verify_response.usage.output_tokens_details.reasoning_tokens}")
print(f"\n💡 High reasoning tokens = the model is 'thinking hard' about edge cases")

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ THE VERIFICATION RESULT                                                     │
# │                                                                             │
# │ This JSON should include:                                                   │
# │   - has_issues: Did it find any problems?                                   │
# │   - issues_found: What specific bugs or concerns?                           │
# │   - fixed_code: Corrected version with proper error handling                │
# │   - improvements_made: What was improved?                                   │
# └─────────────────────────────────────────────────────────────────────────────┘
print(f"\nVerification Result:\n{verify_response.output_text}")


STEP 2: Verify and Fix (High Effort)
Reasoning tokens: 467

Verification Result:
{
  "has_issues": true,
  "issues_found": [
    "No handling for divisor == 0 (raises ZeroDivisionError without a clear message/context).",
    "No validation that 'numbers' is an iterable (passing None or a non-iterable raises a less-informative TypeError).",
    "No validation that elements of 'numbers' are numeric/divisible by the divisor (TypeError occurs without index/context).",
    "No validation that 'divisor' is a valid numeric value (e.g., None leads to a TypeError without a clear message)."
  ],
  "fixed_code": "def divide_list(numbers, divisor):\n    \"\"\"Return a new list with each item in 'numbers' divided by 'divisor'.\"\"\"\n    if divisor is None:\n        raise TypeError(\"divisor must be a number, not None\")\n\n    # Provide a clearer error for the most common edge case.\n    if divisor == 0:\n        raise ZeroDivisionError(\"divisor must not be zero\")\n\n    try:\n        iterator 

---
## Summary: Choosing the Right Pattern

| Pattern | Use When | Trade-off |
|---------|----------|-----------|
| **Fast Path, Then Escalate** | Mixed difficulty, user-facing apps | Lower average cost, variable latency |
| **Always Reason** | High-risk, compliance, financial | Higher cost, predictable quality |
| **Draft, Then Verify** | Content generation, code review | Two calls, but catches errors |

### Cost Optimization Tips

1. **Monitor confidence scores** — Track when escalation happens to tune thresholds
2. **Batch similar requests** — Group high-risk requests together for predictable billing
3. **Cache reasoning results** — If the same complex query comes in, reuse the high-effort response
4. **Set token budgets** — Use `max_output_tokens` to prevent runaway costs in any pattern

In [60]:
# =============================================================================
# FINAL SUMMARY: Compare Token Usage Across All Patterns
# =============================================================================
# This table shows the dramatic difference between low and high effort.
#
# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ WHAT TO LOOK FOR IN THIS TABLE:                                             │
# │                                                                             │
# │ 1. LOW effort tasks should have FEW or ZERO reasoning tokens                │
# │    → The model didn't need to "think hard"                                  │
# │    → Fast and cheap                                                         │
# │                                                                             │
# │ 2. HIGH effort tasks should have MANY reasoning tokens                      │
# │    → The model "thought carefully" about the problem                        │
# │    → Slower and more expensive, but more thorough                           │
# │                                                                             │
# │ 3. Rows marked with ⚡ = reasoning tokens were actually used                │
# │    → These are the tasks where deep thinking occurred                       │
# └─────────────────────────────────────────────────────────────────────────────┘

print("="*60)
print("TOKEN USAGE COMPARISON ACROSS ALL PATTERNS")
print("="*60)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ COLLECT ALL RESPONSES FROM THE NOTEBOOK                                     │
# │                                                                             │
# │ We stored the API responses in variables so we can compare them here.       │
# │ Each response contains:                                                     │
# │   - response.output_text = the answer the model gave                        │
# │   - response.usage.output_tokens = total tokens in the output               │
# │   - response.usage.output_tokens_details.reasoning_tokens = hidden thinking │
# └─────────────────────────────────────────────────────────────────────────────┘
comparisons = [
    # Pattern 1: Escalation results
    ("Pattern 1: Complex (Low attempt)", result_complex['low_response']),
    ("Pattern 1: Simple (Low - no escalate)", result_simple['low_response']),
    
    # Pattern 2: Financial analysis - always uses high effort
    ("Pattern 2: Financial (High)", response_financial),
    
    # Pattern 3: Code generation - low for draft, high for verify
    ("Pattern 3: Code Draft (Low)", draft_response),
    ("Pattern 3: Code Verify (High)", verify_response),
]

# Add the escalated response if it exists
if result_complex['escalated'] and result_complex['high_response']:
    comparisons.insert(1, ("Pattern 1: Complex (High escalated)", result_complex['high_response']))

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ PRINT THE COMPARISON TABLE                                                  │
# └─────────────────────────────────────────────────────────────────────────────┘
print(f"\n{'Task':<40} {'Reasoning':>12} {'Total':>10}")
print("-"*65)

for name, resp in comparisons:
    # Extract token counts from the response
    reasoning = resp.usage.output_tokens_details.reasoning_tokens  # Hidden thinking
    total = resp.usage.output_tokens                               # All output tokens
    
    # Add a ⚡ marker if reasoning was used (makes it easy to spot)
    marker = "  ⚡" if reasoning > 0 else ""
    print(f"{name:<40} {reasoning:>12} {total:>10}{marker}")

print("-"*65)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ KEY TAKEAWAYS                                                               │
# └─────────────────────────────────────────────────────────────────────────────┘
print("\n" + "="*60)
print("KEY TAKEAWAYS")
print("="*60)
print("""
1. 📈 HIGH EFFORT = MORE REASONING TOKENS
   Look at the ⚡ rows - these used significant reasoning tokens.
   The model "thought harder" on these tasks.

2. 💰 ROUTING SAVES MONEY
   Pattern 1 shows how simple questions skip escalation.
   You only pay for high effort when truly needed.

3. 🎯 THREE PATTERNS FOR DIFFERENT NEEDS:
   • Pattern 1: Escalate → Try cheap first, upgrade if uncertain
   • Pattern 2: Always High → Critical decisions, no shortcuts
   • Pattern 3: Draft+Verify → Fast creation, thorough review

4. ⚡ = Deep thinking occurred
   Rows without ⚡ = quick answers, low cost.
""")

TOKEN USAGE COMPARISON ACROSS ALL PATTERNS

Task                                        Reasoning      Total
-----------------------------------------------------------------
Pattern 1: Complex (Low attempt)                   67        605  ⚡
Pattern 1: Complex (High escalated)               110        686  ⚡
Pattern 1: Simple (Low - no escalate)               0          9
Pattern 2: Financial (High)                       111        620  ⚡
Pattern 3: Code Draft (Low)                         0         26
Pattern 3: Code Verify (High)                     467        873  ⚡
-----------------------------------------------------------------

KEY TAKEAWAYS

1. 📈 HIGH EFFORT = MORE REASONING TOKENS
   Look at the ⚡ rows - these used significant reasoning tokens.
   The model "thought harder" on these tasks.

2. 💰 ROUTING SAVES MONEY
   Pattern 1 shows how simple questions skip escalation.
   You only pay for high effort when truly needed.

3. 🎯 THREE PATTERNS FOR DIFFERENT NEEDS:
   • Pattern 